In [2]:
# go to the root directory
%cd../
import os

assert os.path.exists('./trainers/trainer.py')

D:\repos\701-simplicity-bias


In [32]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from models import generate_fcn
from trainers import Trainer
from synthetic_data.linear_slabs import load_arrays
import datetime
from logging import Logger, StreamHandler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# follow the config of original paper

model_config = dict(
    num_layers=2,
    input_dim=50,
    output_dim=2,
    latent_dim=100,
    use_bn=False,
    dropout_probability=0.0,
    linear_init=None
)

In [21]:
model = generate_fcn(**model_config)
model

Sequential(
  (0): Linear(in_features=50, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=2, bias=True)
)

In [43]:
# use lms_7_noisy
train_data, w = load_arrays('./data/lms_7_noisy_train.npz')
val_data, _ = load_arrays('./data/lms_7_noisy_val.npz')

In [57]:
trainer_logger = Logger('trainer')
trainer_logger.setLevel('DEBUG')
trainer_logger.addHandler(StreamHandler())
trainer_config = dict(
    train_data=dict(
        dataset=train_data,
        batch_size=256,
        shuffle=True
    ), val_data=dict(
        dataset=val_data,
        batch_size=256,
        shuffle=False
    ),
    model=model,
    loss=torch.nn.CrossEntropyLoss(),
    device='cuda' if torch.cuda.is_available() else 'cpu',
    evaluate_interval=500,
    save_interval=0,
    work_dir='./training_logs/lms7_noisy_{}/'.format(datetime.datetime.now().strftime('%m%d%H%M')),
    loss_eps=1e-2,
    logger=trainer_logger,
    max_steps=100000,
    optimizer=dict(
        cls='SGD',
        lr=0.1,
        weight_decay=5.0e-5
    )
)
trainer = Trainer(**trainer_config)

model: Sequential(
  (0): Linear(in_features=50, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=2, bias=True)
)


In [58]:
trainer.run()

Step 0: Loss 0.0005062478012405336
Evaluating ...
Step: 0
Train/SumLoss: 0.18283148643240565
Train/Accuracy: 1.0
Val/SumLoss: 0.3493556591565721
Val/Accuracy: 0.9963
Step 500: Loss 0.00047561159590259194
Evaluating ...
Step: 500
Train/SumLoss: 0.18305313852033578
Train/Accuracy: 1.0
Val/SumLoss: 0.34945420126314275
Val/Accuracy: 0.9962
Step 1000: Loss 0.000320620572892949
Evaluating ...
Step: 1000
Train/SumLoss: 0.18405432988947723
Train/Accuracy: 1.0
Val/SumLoss: 0.3755307869578246
Val/Accuracy: 0.9961
Step 1500: Loss 0.0009318586671724916
Evaluating ...
Step: 1500
Train/SumLoss: 0.18406245238293195
Train/Accuracy: 1.0
Val/SumLoss: 0.3341936993820127
Val/Accuracy: 0.9963
Step 2000: Loss 0.00018836009257938713
Evaluating ...
Step: 2000
Train/SumLoss: 0.17863637276605004
Train/Accuracy: 1.0
Val/SumLoss: 0.34617872221861035
Val/Accuracy: 0.9962
Step 2500: Loss 0.0003685743431560695
Evaluating ...
Step: 2500
Train/SumLoss: 0.17727507303789025
Train/Accuracy: 1.0
Val/SumLoss: 0.35734209537

KeyboardInterrupt: 